### Wandb 예제

#### init

In [5]:
import wandb

# !wandb login
wandb.init(project='MLOps_Interview_CIFAR10_Classification',
           name='Train_2'
           )

#### config 설정

In [6]:
learning_rate = 0.0001
epochs = 20
batch_size = 16

args = {
    "learning_rate": learning_rate,
    "epochs": epochs,
    "batch_size": batch_size
}
wandb.config.update(args)

#### datasets

In [7]:
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import transforms

# Define transformation
transform = transforms.Compose([transforms.ToTensor()])

# Download training data from open datasets.
full_training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transform,
)

# Split training data into train and validation sets
train_size = int(0.8 * len(full_training_data))
valid_size = len(full_training_data) - train_size
training_data, validation_data = random_split(full_training_data, [train_size, valid_size])

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transform,
)

#### dataloader

In [8]:
# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=args['batch_size'])
valid_dataloader = DataLoader(validation_data, batch_size=args['batch_size'])
test_dataloader = DataLoader(test_data, batch_size=args['batch_size'])

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([16, 1, 28, 28])
Shape of y: torch.Size([16]) torch.int64


#### cuda

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


#### network

In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


#### cost function, optimizer

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=args['learning_rate'])

#### def train, test

In [12]:

def train(dataloader, model, loss_fn, optimizer, epoch):
    size = len(dataloader.dataset)
    model.train()
    total_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    wandb.log({"train_loss": total_loss / len(dataloader)}, step=epoch)

def validate(dataloader, model, loss_fn, epoch):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    valid_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            valid_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    valid_loss /= num_batches
    correct /= size
    print(f"Validation Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {valid_loss:>8f} \n")
    wandb.log({"valid_loss": valid_loss, "valid_acc": correct}, step=epoch)
    
def test(dataloader, model, loss_fn, epoch):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    wandb.log({"test_loss": test_loss, "test_acc": correct}, step=epoch)

#### train

In [13]:
epochs = args['epochs']
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, t)
    validate(valid_dataloader, model, loss_fn, t)
    test(test_dataloader, model, loss_fn, t)
print("Done!")

Epoch 1
-------------------------------
loss: 2.305412  [    0/48000]
loss: 2.292319  [ 1600/48000]
loss: 2.292901  [ 3200/48000]
loss: 2.288993  [ 4800/48000]
loss: 2.293443  [ 6400/48000]
loss: 2.281941  [ 8000/48000]
loss: 2.294272  [ 9600/48000]
loss: 2.302605  [11200/48000]
loss: 2.272305  [12800/48000]
loss: 2.305001  [14400/48000]
loss: 2.274620  [16000/48000]
loss: 2.290270  [17600/48000]
loss: 2.283370  [19200/48000]
loss: 2.287082  [20800/48000]
loss: 2.290796  [22400/48000]
loss: 2.275784  [24000/48000]
loss: 2.287327  [25600/48000]
loss: 2.276525  [27200/48000]
loss: 2.280521  [28800/48000]
loss: 2.272375  [30400/48000]
loss: 2.268008  [32000/48000]
loss: 2.281464  [33600/48000]
loss: 2.269574  [35200/48000]
loss: 2.273771  [36800/48000]
loss: 2.269223  [38400/48000]
loss: 2.274923  [40000/48000]
loss: 2.277888  [41600/48000]
loss: 2.266937  [43200/48000]
loss: 2.261793  [44800/48000]
loss: 2.260916  [46400/48000]
Test Error: 
 Accuracy: 33.6%, Avg loss: 2.259726 

Epoch 2


#### save pth

In [14]:
save_dir = './models/'

torch.save(model.state_dict(), save_dir + "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


#### model

In [15]:
model = NeuralNetwork()
model.load_state_dict(torch.load(save_dir + "model.pth"))

<All keys matched successfully>

#### evaluation

In [16]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
